# 네이버 플레이스 크롤링 파이프라인
1. 웹드라이버 초기화
- 실제 웹 브라우저를 자동으로 실행하여 웹 페이지와 상호작용
- 이 과정에서 브라우저의 크기와 기타 옵션들을 설정하고, 웹 페이지가 로드되기 전까지 기다리는 시간을 설정

2. 가게명 추출
- 주어진 웹 페이지에서 가게명을 추출
- 웹 페이지의 HTML 소스를 BeautifulSoup을 이용해 파싱하고, 해당 가게명이 포함된 태그를 찾아서 텍스트를 추출
- 추출된 가게명은 이후 리뷰 데이터와 함께 저장될 파일의 이름으로 사용

3. 리뷰 크롤링
- 페이지를 스크롤하여 리뷰 목록을 점차적으로 로딩하고, '더보기' 버튼을 클릭하여 추가적인 리뷰를 불러옴
- 각 리뷰는 특정 HTML 태그에서 추출되며, 최대 크롤링할 리뷰 수(max_reviews)를 설정하여 필요한 만큼만 수집

리뷰 데이터 저장
네 번째 단계는 수집한 리뷰 데이터를 파일로 저장하는 것입니다. 저장되는 파일은 CSV 형식으로, 각 리뷰의 내용과 해당하는 가게명이 함께 기록됩니다. 이 파일은 나중에 분석이나 처리에 사용될 수 있습니다.

여러 링크에서 리뷰 수집
마지막 단계는 여러 가게 또는 링크에서 리뷰를 수집하는 과정입니다. 링크 리스트를 순차적으로 처리하며, 각 링크에서 가게명과 리뷰를 추출하여 저장합니다. 이 단계는 여러 가게에서 리뷰를 한 번에 크롤링하고자 할 때 유용합니다.

## 1. 라이브러리 임포트

In [3]:
import time
import datetime
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

## 2. 웹드라이버 초기화

In [7]:
def initialize_webdriver():
    options = webdriver.ChromeOptions()
    options.add_argument('window-size=1920x1080')
    options.add_argument("disable-gpu")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.implicitly_wait(10)
    return driver

## 3. 가게명 크롤링

In [9]:
def extract_store_name(driver):
    try:
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        store_name_tag = soup.select_one('div#_title.LylZZ.v8v5j > div > span.GHAhO')
        return store_name_tag.text.strip() if store_name_tag else "Unknown Store"
    except Exception as e:
        print(f"가게명 추출 중 오류 발생: {e}")
        return "Unknown Store"

## 4. 리뷰 크롤링

In [63]:
def extract_reviews(driver, max_reviews=100):
    try:
        reviews = []
        print("리뷰 수집 시작...")

        # 페이지 스크롤 및 '더보기' 버튼 클릭
        for _ in range(30):  # 최대 30번 스크롤
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
            time.sleep(0.5)  # 스크롤 후 대기
            try:
                more_button = WebDriverWait(driver, 2).until(
                    EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a'))
                )
                more_button.click()
                print("더보기 버튼 클릭!")
                time.sleep(0.5)  # 버튼 클릭 후 대기
            except Exception:
                print("더보기 버튼이 더 이상 없습니다.")
                break

        # 페이지 HTML 파싱
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')

        # 리뷰 데이터 추출
        review_elements = soup.select('li.pui__X35jYm.EjjAW')

        if not review_elements:  # 리뷰 데이터가 없을 경우 처리
            print("리뷰가 존재하지 않습니다.")
            return []

        for element in review_elements:
            # 리뷰 본문 크롤링
            content_tag = element.select_one('div.pui__vn15t2 > a[role="button"]')
            content = content_tag.text.strip() if content_tag else ""
            if content:
                reviews.append(content)
                print(f"리뷰 수집: {content}")
            if len(reviews) >= max_reviews:  # 최대 리뷰 수 도달 시 종료
                print(f"리뷰 {max_reviews}개 수집 완료.")
                break

        return reviews

    except Exception as e:
        print(f"리뷰 크롤링 중 오류 발생: {e}")
        return []

In [65]:
def save_reviews_to_csv(reviews, store_name):
    try:
        now = datetime.datetime.now()
        file_name = f'{store_name}_네이버플레이스리뷰.csv'
        with open(file_name, mode='w', encoding='utf-8-sig', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['store_name', 'content'])  # 헤더 작성
            for review in reviews:
                writer.writerow([store_name, review])
        print(f"리뷰 데이터 저장 완료: {file_name}")
    except Exception as e:
        print(f"리뷰 저장 중 오류 발생: {e}")

## 5. 네이버 플레이스 접속 및 실행

In [60]:
def scrape_reviews_from_naver(driver, place_url, max_reviews=100):
    try:
        print("네이버 플레이스 접속 중...")
        driver.get(place_url)
        time.sleep(3)
        print("네이버 플레이스 접속 완료!")

        # 가게명 크롤링
        store_name = extract_store_name(driver)
        print(f"가게명: {store_name}")

        # 리뷰 크롤링
        reviews = extract_reviews(driver, max_reviews)
        if reviews:
            print(f"리뷰 {len(reviews)}개 크롤링 완료!")
            save_reviews_to_csv(reviews, store_name)
        else:
            print("리뷰 크롤링 결과 없음.")

    except Exception as e:
        print(f"오류 발생: {e}")

## 6. 여러 링크에서 리뷰 수집

In [70]:
def scrape_reviews_from_multiple_links(driver, links, max_reviews=100):
    all_reviews = []  # 모든 링크에서 수집된 리뷰를 저장할 리스트
    try:
        for link in links:
            print(f"'{link}'에 대해 리뷰 수집 시작...")

            # 해당 링크로 이동
            driver.get(link)
            time.sleep(3)
            print(f"'{link}' 접속 완료!")

            # 가게명 크롤링
            store_name = extract_store_name(driver)
            print(f"가게명: {store_name}")

            # 리뷰 크롤링
            reviews = extract_reviews(driver, max_reviews)
            if reviews:
                print(f"리뷰 {len(reviews)}개 크롤링 완료!")
                all_reviews.append((store_name, reviews))
            else:
                print(f"리뷰 크롤링 결과 없음.")

        # 모든 리뷰 저장
        save_all_reviews_to_csv(all_reviews)

    except Exception as e:
        print(f"오류 발생: {e}")

## 7. 리뷰 데이터 저장 (.csv)

In [72]:
# 모든 리뷰 데이터를 하나의 CSV 파일로 저장
def save_all_reviews_to_csv(all_reviews):
    try:
        now = datetime.datetime.now()
        file_name = f"모든_리뷰_{now.strftime('%Y%m%d_%H%M%S')}.csv"
        with open(file_name, mode='w', encoding='utf-8-sig', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['store_name', 'content'])  # 헤더 작성
            for store_name, reviews in all_reviews:
                for review in reviews:
                    writer.writerow([store_name, review])
        print(f"모든 리뷰 데이터 저장 완료: {file_name}")
    except Exception as e:
        print(f"모든 리뷰 저장 중 오류 발생: {e}")

# 메인 실행

In [75]:
# 메인 실행
if __name__ == "__main__":
    driver = initialize_webdriver()
    try:
        # 여러 링크 리스트
        links = [
            'https://m.place.naver.com/restaurant/1478636575/review/visitor?reviewSort=recent',
            'https://m.place.naver.com/restaurant/1533608222/review/visitor?reviewSort=recent',
            # 추가 링크...
        ]

        # 여러 링크에서 리뷰 수집
        scrape_reviews_from_multiple_links(driver, links, max_reviews=100)

    except Exception as e:
        print(f"오류 발생: {e}")
    finally:
        driver.quit()
        print("브라우저 종료 완료.")

'https://m.place.naver.com/restaurant/1478636575/review/visitor?reviewSort=recent'에 대해 리뷰 수집 시작...
'https://m.place.naver.com/restaurant/1478636575/review/visitor?reviewSort=recent' 접속 완료!
가게명: 쿳사 연희
리뷰 수집 시작...
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼 클릭!
더보기 버튼이 더 이상 없습니다.
리뷰 수집: 쿳사 세 번째 방문인데 시즌별로 메뉴가 바뀌는지 몰랐어요ㅎㅎ 기대도 안했다가 새로운 메뉴들 먹게 되었는데 역시나 특색있고 맛있었습니다! 음식이 색이 다채로워서 사진으로 찍어도 예쁘고 맛도 흔하지 않아서 넘 좋았어요. 특히 화이트 라구 파스타 넘넘 맛있었습니다~ 항상 좋은 기억 제공해 주셔서 감사해요!
리뷰 수집: 재료도 신선했고, 너무 맛있었어요~
리뷰 수집: 사장님이 엄청 친절하시고 무엇보다 마싯어요 //예약 실수했는데 잘 안내해 주셔서 무사히 식사했습니다. 너무 감사해요. 또 방문하겠슴다 🤍🤍
리뷰 수집: 가성비, 음식 퀄리티, 친절도 모두 대만족. 강추합니다!
리뷰 수집: 플레이팅도 너무 예쁘고, 맛있게 잘 먹었습니다:)
리뷰 수집: 쿳사의 시즌 메뉴 늘 기대됩니다. 너무 맛있어요.
리뷰 수집: 2번째 간 곳인데 또 재방문의사 200%입니다.너무너무 맛있고너무너무 예쁘고너무너무 친절하고 좋은 시간 보내고 왔어요.먹은 것 중에 특히 크랩